In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
processed_data = glueContext.create_dynamic_frame.from_catalog(database="stocks", table_name="processed-data")

In [ ]:
processed_data.printSchema()

In [ ]:
processed_data.toDF().createOrReplaceTempView("temp_processed_data")

# Running the SQL statement which 
sortedDF = spark.sql("select * from temp_processed_data order by datetime ASC")
print(f'Running (count): {sortedDF.count()}')

sortedDF.show(5)

In [ ]:
from awsglue.dynamicframe import DynamicFrame

sortedDF = DynamicFrame.fromDF(sortedDF, glueContext, "sortedDF")

In [ ]:
sortedDF = sortedDF.coalesce(1)

In [ ]:
try:
    datasink = glueContext.write_dynamic_frame.from_options(
        frame = sortedDF, connection_type="s3",
        connection_options = {"path": "s3://villamil-stocks-project/data/sorted-data/"},
        format = "csv")
    print('Transformed data written to S3')
except Exception as ex:
    print('Something went wrong')
    print(ex)